In [73]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import community
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch_geometric.nn import Node2Vec
from torch_geometric.utils.convert import from_networkx
from torch_geometric.utils import degree, to_networkx
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import seaborn as sns
import pickle
sns.set()

In [74]:
embeddings_1 = pd.read_json('embeddings_with_cluster1.json',lines=True)
embeddings_1.head()
embeddings_2 = pd.read_json('embeddings_with_cluster2.json',lines=True)
embeddings_2.head()
embeddings_3 = pd.read_json('embeddings_with_cluster3.json',lines=True)
embeddings_3.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_19972\436348439.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  embeddings_1 = pd.read_json('embeddings_with_cluster1.json',lines=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_19972\436348439.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  embeddings_1 = pd.read_json('embeddings_with_cluster1.json',lines=True)
C:\Users\HP\AppData\Local\Temp\ipykernel_19972\436348439.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,cluster_label
0,-0.049728,0.220807,-0.009551,-0.260663,0.061310,0.031595,-0.149981,0.082681,-0.009008,0.039538,...,0.077505,0.160976,-0.041870,0.289285,-0.138164,-0.098474,0.041500,0.104904,0.175890,2
1,0.151118,0.167657,0.081400,-0.098236,0.169692,0.039468,-0.045907,0.097608,0.106262,0.014770,...,-0.095225,-0.127120,0.037120,0.039927,-0.015724,0.232021,-0.162908,0.086416,0.187216,2
2,0.404190,-0.269846,0.082369,0.156145,-0.057768,0.189231,-0.005160,0.101423,-0.240075,0.043559,...,-0.139599,-0.355075,0.384469,0.124133,0.036653,-0.125443,0.153743,0.325689,-0.356468,2
3,0.017481,0.048374,-0.077696,-0.417364,0.004395,-0.183351,0.104363,0.192619,0.220743,0.220823,...,-0.132920,-0.194306,0.093333,-0.052717,-0.032724,-0.241660,0.055065,0.093820,0.044434,-1
4,0.069057,-0.209454,-0.048993,0.038812,-0.070603,0.002607,0.179880,-0.093794,0.033769,-0.056118,...,-0.163917,-0.072509,0.098020,-0.157713,0.008254,-0.054138,0.184190,0.102305,-0.113230,2


In [21]:
anomalies_full_df_1 = pd.read_csv('final_upd_3.csv')
anomalies_full_df_1.drop('Unnamed: 0', axis =1, inplace = True)
anomalies_full_df = pd.read_csv('final_upd_2.csv')
anomalies_full_df.drop('Unnamed: 0', axis =1, inplace = True)
df_test = anomalies_full_df[['cust_id','betweenness_centrality','degree','eigenvector_centrality','pagerank']]
for column in ['betweenness_centrality', 'degree', 'eigenvector_centrality', 'pagerank']:
    df_test[f'{column}_rank'] = df_test[column].rank(method='average',ascending = False)
df_test.head(1)
df_test_sorted = df_test.sort_values(by=['degree_rank', 'betweenness_centrality_rank', 'eigenvector_centrality_rank', 'pagerank_rank'], ascending=True)
# High betweenness centrality nodes
high_betweenness_nodes = df_test[df_test['betweenness_centrality'] > df_test['betweenness_centrality'].quantile(0.9)]
# Normalizing centrality measures
for col in ['betweenness_centrality', 'degree', 'eigenvector_centrality', 'pagerank']:
    df_test[f'{col}_norm'] = (df_test[col] - df_test[col].min()) / (df_test[col].max() - df_test[col].min())

# Creating a composite score
df_test['composite_brokerage_score'] = df_test[['betweenness_centrality_norm', 'degree_norm', 'eigenvector_centrality_norm', 'pagerank_norm']].mean(axis=1)

# Identifying top brokerage nodes based on the composite score
top_brokerage_nodes = df_test.sort_values(by='composite_brokerage_score', ascending=False)
top_brokerage_nodes.head(1)

C:\Users\HP\AppData\Local\Temp\ipykernel_19972\2560387486.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{column}_rank'] = df_test[column].rank(method='average',ascending = False)
C:\Users\HP\AppData\Local\Temp\ipykernel_19972\2560387486.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[f'{column}_rank'] = df_test[column].rank(method='average',ascending = False)
C:\Users\HP\AppData\Local\Temp\ipykernel_19972\2560387486.py:7: SettingWithCopyWarning: 
A value is trying to be set on a 

,cust_id,betweenness_centrality,degree,eigenvector_centrality,pagerank,betweenness_centrality_rank,degree_rank,eigenvector_centrality_rank,pagerank_rank,betweenness_centrality_norm,degree_norm,eigenvector_centrality_norm,pagerank_norm,composite_brokerage_score
188082,CUST36405209,0.000167,20.0,0.510221,0.000142,8455.0,4778.5,1.0,1.0,0.178731,0.512821,1.0,1.0,0.672888


In [22]:
top_1_percent_cutoff = top_brokerage_nodes['composite_brokerage_score'].quantile(0.999)  # Adjust quantile for different cutoffs
highly_influential_nodes = top_brokerage_nodes[top_brokerage_nodes['composite_brokerage_score'] >= top_1_percent_cutoff]

In [23]:
with open('cust_id_to_index.pkl', 'rb') as f:
    cust_id_to_index = pickle.load(f)

In [105]:
transactions_combined = pd.read_csv('transactions_combined_up.csv').drop('Unnamed: 0',axis = 1)

In [106]:
transactions_combined['sender_idx'] = transactions_combined['cust_id_sender'].map(cust_id_to_index)
transactions_combined['receiver_idx'] = transactions_combined['cust_id_receiver'].map(cust_id_to_index)

In [107]:
transactions_combined.head(1)

,cust_id_sender,cust_id_receiver,amount,count,sender_idx,receiver_idx
0,CUST10000513,CUST13934055,360.0,1,118080,76636


In [108]:
anomalies_df = anomalies_full_df_1[anomalies_full_df_1.anomaly == -1]
df_merged = anomalies_df.merge(highly_influential_nodes[['cust_id','composite_brokerage_score']], how = 'inner', on = 'cust_id')
influential_customers = df_merged.cust_id.tolist()

In [109]:
influential_mappings = {cust_id: index for cust_id, index in cust_id_to_index.items() if cust_id in influential_customers}
influential_df = pd.DataFrame(list(influential_mappings.items()), columns=['cust_id', 'index'])
influential_df

,cust_id,index
0,CUST34310812,224
1,CUST29461884,257
2,CUST53667999,341
3,CUST52962125,404
4,CUST97925988,1102
...,...,...
188,CUST99278393,133196
189,CUST98291483,138265
190,CUST99675035,138458
191,CUST34886918,150698


In [101]:
import torch
import pandas as pd


# Convert embeddings DataFrame to a PyTorch tensor for efficient computation
embeddings_tensor = torch.tensor(embeddings_1.values).float()

# Initialize a list to store data about influential customers and their top similar nodes
data = {'influential_cust_id': [], 'similar_cust_ids': []}

# Iterate over influential customers
for cust_id in influential_customers:
    # Get the index of the current influential customer from the cust_id_to_index mapping
    index = cust_id_to_index[cust_id]
    
    # Extract the embedding for the current influential customer
    influential_embedding = embeddings_tensor[index].unsqueeze(0)  # Add batch dimension for cosine similarity calculation
    
    # Calculate cosine similarity
    normalized_embeddings = torch.nn.functional.normalize(embeddings_tensor, p=2, dim=1)
    normalized_influential_embedding = torch.nn.functional.normalize(influential_embedding, p=2, dim=1)
    similarity_scores = torch.mm(normalized_influential_embedding, normalized_embeddings.transpose(0, 1))
    
    # Set the self-similarity to a low value to exclude it from top results
    similarity_scores[0, index] = -1
    # Find indices of the top 5 similar nodes, excluding the highest one as it's the node itself
    top_similar_indices = torch.topk(similarity_scores, k=3, largest=True).indices[0][1:]

    # Convert indices back to customer IDs using a list comprehension and the cust_id_to_index mapping
    top_similar_cust_ids = [cust_id for cust_id, idx in cust_id_to_index.items() if idx in top_similar_indices.tolist()]
    
    # Append the results to our data structure
    data['influential_cust_id'].append(cust_id)
    data['similar_cust_ids'].append(top_similar_cust_ids)

# Convert the data into a DataFrame
relationships_df = pd.DataFrame(data)

# If you prefer each similar customer ID to be in separate rows:
relationships_df_exploded = relationships_df.explode('similar_cust_ids').reset_index(drop=True)

print(relationships_df)
print(relationships_df_exploded)

    influential_cust_id                  similar_cust_ids
0          CUST98901965      [CUST35476414, CUST17046649]
1          CUST71268159      [CUST41167081, CUST85018083]
2          CUST94760680  [EXTERNAL861334, EXTERNAL884936]
3          CUST26911489  [EXTERNAL431313, EXTERNAL730236]
4          CUST48605422      [CUST37416122, CUST57301076]
..                  ...                               ...
188        CUST73212501    [EXTERNAL165458, CUST63538915]
189        CUST36405209  [EXTERNAL934888, EXTERNAL304920]
190        CUST17148894      [CUST12980531, CUST20671743]
191        CUST41581393    [CUST42393992, EXTERNAL540048]
192        CUST28168926      [CUST97630737, CUST34893846]

[193 rows x 2 columns]
    influential_cust_id similar_cust_ids
0          CUST98901965     CUST35476414
1          CUST98901965     CUST17046649
2          CUST71268159     CUST41167081
3          CUST71268159     CUST85018083
4          CUST94760680   EXTERNAL861334
..                  ...           

In [102]:
unique_values = pd.unique(relationships_df_exploded[['influential_cust_id', 'similar_cust_ids']].values.ravel('K'))

In [130]:
import torch
import pandas as pd

# Convert embeddings DataFrame to a PyTorch tensor for efficient computation
embeddings_tensor = torch.tensor(embeddings_2.values).float()

# Initialize a list to store data about influential customers and their top similar nodes
data = {'influential_cust_id': [], 'similar_cust_ids': []}

# Iterate over influential customers
for cust_id in influential_customers:
    # Get the index of the current influential customer from the cust_id_to_index mapping
    index = cust_id_to_index[cust_id]
    
    # Extract the embedding for the current influential customer
    influential_embedding = embeddings_tensor[index].unsqueeze(0)  # Add batch dimension for cosine similarity calculation
    
    # Calculate cosine similarity
    normalized_embeddings = torch.nn.functional.normalize(embeddings_tensor, p=2, dim=1)
    normalized_influential_embedding = torch.nn.functional.normalize(influential_embedding, p=2, dim=1)
    similarity_scores = torch.mm(normalized_influential_embedding, normalized_embeddings.transpose(0, 1))
    
    # Set the self-similarity to a low value to exclude it from top results
    similarity_scores[0, index] = -1
    # Find indices of the top 5 similar nodes, excluding the highest one as it's the node itself
    top_similar_indices = torch.topk(similarity_scores, k=3, largest=True).indices[0][1:]

    # Convert indices back to customer IDs using a list comprehension and the cust_id_to_index mapping
    top_similar_cust_ids = [cust_id for cust_id, idx in cust_id_to_index.items() if idx in top_similar_indices.tolist()]
    
    # Append the results to our data structure
    data['influential_cust_id'].append(cust_id)
    data['similar_cust_ids'].append(top_similar_cust_ids)

# Convert the data into a DataFrame
relationships_df_2 = pd.DataFrame(data)

relationships_df_exploded_2 = relationships_df.explode('similar_cust_ids').reset_index(drop=True)

In [131]:
unique_values_2 = pd.unique(relationships_df_exploded_2[['influential_cust_id', 'similar_cust_ids']].values.ravel('K'))

In [132]:
import torch
import pandas as pd

# Convert embeddings DataFrame to a PyTorch tensor for efficient computation
embeddings_tensor = torch.tensor(embeddings_3.values).float()

# Initialize a list to store data about influential customers and their top similar nodes
data = {'influential_cust_id': [], 'similar_cust_ids': []}

# Iterate over influential customers
for cust_id in influential_customers:
    # Get the index of the current influential customer from the cust_id_to_index mapping
    index = cust_id_to_index[cust_id]
    
    # Extract the embedding for the current influential customer
    influential_embedding = embeddings_tensor[index].unsqueeze(0)  # Add batch dimension for cosine similarity calculation
    
    # Calculate cosine similarity
    normalized_embeddings = torch.nn.functional.normalize(embeddings_tensor, p=2, dim=1)
    normalized_influential_embedding = torch.nn.functional.normalize(influential_embedding, p=2, dim=1)
    similarity_scores = torch.mm(normalized_influential_embedding, normalized_embeddings.transpose(0, 1))
    
    # Set the self-similarity to a low value to exclude it from top results
    similarity_scores[0, index] = -1
    # Find indices of the top 5 similar nodes, excluding the highest one as it's the node itself
    top_similar_indices = torch.topk(similarity_scores, k=3, largest=True).indices[0][1:]

    # Convert indices back to customer IDs using a list comprehension and the cust_id_to_index mapping
    top_similar_cust_ids = [cust_id for cust_id, idx in cust_id_to_index.items() if idx in top_similar_indices.tolist()]
    
    # Append the results to our data structure
    data['influential_cust_id'].append(cust_id)
    data['similar_cust_ids'].append(top_similar_cust_ids)

# Convert the data into a DataFrame
relationships_df_3 = pd.DataFrame(data)

# If you prefer each similar customer ID to be in separate rows:
relationships_df_exploded_3 = relationships_df.explode('similar_cust_ids').reset_index(drop=True)

In [133]:
unique_values_3 = pd.unique(relationships_df_exploded_3[['influential_cust_id', 'similar_cust_ids']].values.ravel('K'))

In [135]:
common_influential = list(set(unique_values) & set(unique_values_2) & set(unique_values_3))

In [110]:
import pandas as pd
import networkx as nx


# Create an empty graph. Use nx.DiGraph() for a directed graph if that's more appropriate for your data.
G = nx.Graph()

# Add edges to the graph from the transactions DataFrame
for _, transaction in transactions_combined.iterrows():
    sender_id = transaction['cust_id_sender']
    receiver_id = transaction['cust_id_receiver']
    amount = transaction['amount']  # Assuming this is how you wish to use the transaction amount, e.g., as edge weight
    
    # Add edge with the amount as edge attribute (weight)
    G.add_edge(sender_id, receiver_id, weight=amount)

In [112]:
#Direct Graphs

In [ ]:
import networkx as nx

# Assuming 'G' is your graph and 'influential_customers' is a list of customer IDs
nodes_to_include = set(common_influential)

for customer in unique_values:
    # Get direct neighbors of 'customer'
    neighbors = set(G.neighbors(customer))
    
    # Add these neighbors to our set
    nodes_to_include.update(neighbors)

# Create the subgraph including all nodes identified (influential customers and their direct neighbors)
subG_direct = G.subgraph(nodes_to_include)

In [ ]:
len(subG_direct)

In [145]:
edges_data = pd.DataFrame(subG_direct.edges(data=True), columns=['cust_id_sender', 'cust_id_receiver', 'amount'])
edges_data.to_csv("subG_direct_edges.csv", index=False)

In [119]:
nodes_to_include_ind = set(common_influential)
distance = 2 # Adjust based on how "far" you want to go in terms of indirect connections

for customer in influential_customers:
    # BFS to get all nodes within 'distance' steps from 'customer'
    bfs_nodes = nx.single_source_shortest_path_length(G, customer,distance)
    
    # Add these nodes to our set
    nodes_to_include_ind.update(bfs_nodes.keys())

# Create the subgraph including all nodes identified
subG_indirect = G.subgraph(nodes_to_include_ind)

In [120]:
len(subG_indirect)

39813

In [146]:
edges_data = pd.DataFrame(subG_indirect.edges(data=True), columns=['cust_id_sender', 'cust_id_receiver', 'amount'])
edges_data.to_csv("subG_indirect_edges.csv", index=False)

In [129]:
edges_data

,cust_id_sender,targecust_id_receivert,amount
0,CUST54801958,CUST43814376,{'weight': 502.0}
1,EXTERNAL261114,CUST67282733,{'weight': 293.5}
2,EXTERNAL118720,CUST83708217,{'weight': 1132.0}
3,EXTERNAL118720,CUST95405296,{'weight': 1134.0}
4,CUST92080684,CUST75674030,{'weight': 12.0}
...,...,...,...
75146,CUST90034020,CUST73418006,{'weight': 959.0}
75147,CUST62924281,CUST54381300,{'weight': 2690.0}
75148,EXTERNAL881945,CUST39444554,{'weight': 290.0}
75149,CUST30991673,EXTERNAL165302,{'weight': 939.0}
